# Image Resize

* 원본 이미지 비율 유지하며 축소
* 남는 영역은 자연스럽게 패딩(배경 이미지와 유사하게)
* 최종 크기: 224×224
* 정규화 (픽셀값 01 or -11)

In [ ]:
import cv2
import numpy as np
import os

model = "train"
name = "grease"

# 파일 개수
folder_path = f'dataset/{model}/{name}'
file_list = os.listdir(folder_path)

if not os.path.exists(f"dataset/{model}/{name}_new"):
    os.makedirs(f"dataset/{model}/{name}_new")

# .jpg 파일만 필터링
jpg_files = len([f for f in file_list if f.endswith('.png')])

for i in range(1, jpg_files + 1):
    img = cv2.imread(f"dataset/{model}/{name}/{name} ({i}).png")

    h, w = img.shape[:2]
    scale = 224 / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)

    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_CUBIC)

    # 패딩 계산
    top = (224 - new_h) // 2
    bottom = 224 - new_h - top
    left = (224 - new_w) // 2
    right = 224 - new_w - left

    padded = cv2.copyMakeBorder(resized, top, bottom, left, right,
                   cv2.BORDER_REPLICATE)

    # 픽셀 정규화 (0~1 사이)
    normalized = padded.astype(np.float32) / 255.0

    # 저장 (선택사항, 훈련 중 바로 넣을 수도 있음)
    save_path = f"dataset/{model}/{name}_new/{name}_{i:05}.jpg"
    cv2.imwrite(save_path, (normalized * 255).astype(np.uint8))  # 저장은 다시 0~255로

# Data Augmentation

In [ ]:
import os
import cv2
import numpy as np
import albumentations as A
from PIL import Image

# 🔧 설정
input_dir = "dataset/train/grease"  # 원본 이미지가 들어 있는 폴더
output_dir = "dataset/train/grease_new"  # 증강된 이미지를 저장할 폴더
os.makedirs(output_dir, exist_ok=True)

# 🔁 증강 파이프라인 정의
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=20, border_mode=cv2.BORDER_REFLECT),
    A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.05, p=0.8),
    A.Affine(translate_percent=(0.05, 0.05), scale=(0.95, 1.05), p=0.8),
])

# 📸 증강 이미지 개수 설정
n_augmented = 5

# 원본 이미지 파일 리스트 (grease_00001.jpg ~ grease_01000.jpg)
file_list = [f"grease_{i:05}.jpg" for i in range(1, 1001)]

# 증강된 이미지 번호를 추적하기 위한 카운터
augmented_counter = 1001

# 각 이미지에 대해 증강 처리
for filename in file_list:
    img_path = os.path.join(input_dir, filename)

    # 이미지 열기 (cv2는 BGR로 불러오기 때문에 RGB로 변환)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 증강 및 저장
    for i in range(1, n_augmented + 1):
        augmented = transform(image=img)['image']
        out_img = Image.fromarray(augmented)

        # 새 파일명 (grease_00001.jpg ~ grease_00005.jpg, grease_00006.jpg ~ grease_00010.jpg, ...)
        new_name = f"grease_{augmented_counter:05}.jpg"
        out_path = os.path.join(output_dir, new_name)
        out_img.save(out_path)

        # 증강된 이미지 번호 증가
        augmented_counter += 1

    print(f"✅ {filename}에 대해 {n_augmented}개의 증강 이미지가 저장되었습니다.")

print(f"✅ 총 {augmented_counter - 1}개의 증강 이미지가 저장되었습니다.")